In [1]:
import pickle

with open('message.pickle', 'wb') as handle:
    pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
import telebot

bot = telebot.TeleBot("5069674898:AAGdHYM4cn-ZqQqbhR2vke34ke8qMos1lTY", parse_mode=None)

WELCOME_MSG = "👋 Howdy, how are you doing? \
I can convert video to the following formats: *.avi, *.gif, *.mov! \
Just upload a short video for me, and then choose file type you want."

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, WELCOME_MSG)

@bot.message_handler(content_types=['video', 'video_note', 'document'])
def handle_docs_video(message):
    # Get file
    if message.content_type == 'video_note':
        file_info = bot.get_file(message.video_note.file_id)
    elif message.content_type == 'video':
        file_info = bot.get_file(message.video.file_id)
    elif message.content_type == 'document':
        file_info = bot.get_file(message.document.file_id)
    
    ## Check if filetype is supported 
    file_type = file_info.file_path.split('.')[-1].lower()
    if (file_type in ['mov', 'mp4']):
        bot.reply_to(message, 'Detected file type: "{}". Congrats! It is supported!'.format(file_type))
    else:
        bot.reply_to(message, 'Detected file type: "{}". Sorry is not supported!'.format(file_type))
    
    ## Save temporary file
    downloaded_file = bot.download_file(file_info.file_path)
    temp_filename = str(file_info.file_path).split('/')[-1]
    with open(temp_filename,'wb') as new_file:
        new_file.write(downloaded_file)
        
    # Convert file
    import ffmpeg
    target_file_type = '.mov'
    stream = ffmpeg.input(temp_filename)
    output_name = '{}.{}'.format(target_file_type.split('.')[0], target_file_type)
    stream = ffmpeg.output(stream, output_name)
    ffmpeg.run(stream)
    
    # Send file
    with open(output_name,'rb') as converted_file:
        bot.send_document(message.chat.id, converted_file)
#     with open('message.pickle', 'wb') as handle:
#         pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)

bot.infinity_polling()

2022-01-19 19:16:50,541 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling exception: [Errno 2] No such file or directory: 'ffmpeg': 'ffmpeg'"
2022-01-19 19:16:50,544 (__init__.py:623 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/Users/evgeni/workspace/bot-converter/venv/lib/python3.6/site-packages/telebot/__init__.py", line 618, in infinity_polling
    allowed_updates=allowed_updates, *args, **kwargs)
  File "/Users/evgeni/workspace/bot-converter/venv/lib/python3.6/site-packages/telebot/__init__.py", line 664, in polling
    self.__threaded_polling(non_stop, interval, timeout, long_polling_timeout, allowed_updates)
  File "/Users/evgeni/workspace/bot-converter/venv/lib/python3.6/site-packages/telebot/__init__.py", line 726, in __threaded_polling
    raise e
  File "/Users/evgeni/workspace/bot-converter/venv/lib/python3.6/site-packages/telebot/__init__.py", line 686, in __threaded_polling
    self.worker_pool.raise_exception

In [22]:
with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [27]:
b.document.file_name

'IMG_2311.MOV'